In [2]:
import pdfplumber
from pdfminer.high_level import extract_text

import io
import requests
import re

import pandas as pd
import concurrent.futures

In [19]:
doc_parlement = "https://www.lachambre.be/FLWB/PDF/55/3197/55K3197001.pdf" 
doc2 = "http://www.lachambre.be/FLWB/PDF/55/3509/55K3509001.pdf"
doc3 = "http://www.lachambre.be/FLWB/PDF/55/2894/55K2894001.pdf"
doc4 = "http://www.lachambre.be/FLWB/PDF/55/2092/55K2092001.pdf"
plenary_doc = "" # 3- Séances Plénières integral - Maciej
com_integral = "" # 4- Commissions integral - Mourad
bulletin = "" # 5- Bulletins des questions et réponses écrites - Ramina
rapport = "" # 6- Rapports déposés en exécution d'une loi - Sarkis

In [2]:
def get_text(url):
    rq = requests.get(url)
    left_column_text_all_pages = ""
    right_column_text_all_pages = ""

    with pdfplumber.open(io.BytesIO(rq.content)) as pdf:
        for page in pdf.pages:
            words = page.extract_words()
            threshold_x = 300

            left_column_words = [word for word in words if word['x0'] < threshold_x]
            right_column_words = [word for word in words if word['x0'] >= threshold_x]

            left_column_text = ' '.join([word['text'] for word in left_column_words])
            right_column_text = ' '.join([word['text'] for word in right_column_words])

            left_column_text_all_pages += left_column_text + ' '
            right_column_text_all_pages += right_column_text + ' '

            left_column_clean = re.sub(r'(?<=\w)-(\s)(?=\w)', r'', left_column_text_all_pages) # remove end_line hyphens
            left_column_clean = re.sub(r'DOC 55 \d{4}/001', '', left_column_clean)
            left_column_clean = re.sub(r'\d{4}/001 DOC 55', '', left_column_clean)
            left_column_clean = re.sub(r'CHAMBRE \d+e SESSION DE LA 55e LÉGISLATURE \d{4}(?: \d+)?', '', left_column_clean)

            right_column_clean = re.sub(r'(?<=\w)-(\s)(?=\w)', r'', right_column_text_all_pages) # remove end_line hyphens
            right_column_clean = re.sub(r'DOC 55 \d{4}/001', '', right_column_clean)
            right_column_clean = re.sub(r'\d{4}/001 DOC 55', '', right_column_clean)
            right_column_clean = re.sub(r'\d{4} KAMER • \d+e ZITTING VAN DE 55e ZITTINGSPERIODE(?: \d+)?', '', right_column_clean)

    return left_column_clean, right_column_clean

    # {"fr_pdf_text" : left_column_text_all_pages, "nl_pdf_text": right_column_text_all_pages}
    

In [14]:
df = pd.read_json('apercu_complet.json')
df_pdf_not_disponible = df[df["DocumentChambre"].str.contains("pasdisponible")]
df_pdf_disponible = df[~df["DocumentChambre"].str.contains("pasdisponible")]

In [17]:
first_300 = df_pdf_disponible.iloc[:300]
first_300[['fr_text', 'nl_text']] = first_300['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))
first_300.to_csv("first_300.csv")

/var/folders/lh/rbpzcs3522qg45v9lc_xvgyc0000gn/T/ipykernel_16533/1449241180.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_300[['fr_text', 'nl_text']] = first_300['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))
/var/folders/lh/rbpzcs3522qg45v9lc_xvgyc0000gn/T/ipykernel_16533/1449241180.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_300[['fr_text', 'nl_text']] = first_300['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))


In [18]:
second_300 = df_pdf_disponible.iloc[301:600]
second_300[['fr_text', 'nl_text']] = second_300['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))
second_300.to_csv("second_300.csv")

/var/folders/lh/rbpzcs3522qg45v9lc_xvgyc0000gn/T/ipykernel_16533/2062434630.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_300[['fr_text', 'nl_text']] = second_300['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))
/var/folders/lh/rbpzcs3522qg45v9lc_xvgyc0000gn/T/ipykernel_16533/2062434630.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_300[['fr_text', 'nl_text']] = second_300['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))


In [20]:
from_600_1300 = df_pdf_disponible.iloc[601:1300]
from_600_1300[['fr_text', 'nl_text']] = from_600_1300['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))
from_600_1300.to_csv("from_600_1300.csv")

/var/folders/lh/rbpzcs3522qg45v9lc_xvgyc0000gn/T/ipykernel_16533/2130033794.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from_600_1300[['fr_text', 'nl_text']] = from_600_1300['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))
/var/folders/lh/rbpzcs3522qg45v9lc_xvgyc0000gn/T/ipykernel_16533/2130033794.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from_600_1300[['fr_text', 'nl_text']] = from_600_1300['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))


In [21]:
from_1300_end = df_pdf_disponible.iloc[1301:]
from_1300_end[['fr_text', 'nl_text']] = from_1300_end['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))
from_1300_end.to_csv("from_1300_end.csv")

/var/folders/lh/rbpzcs3522qg45v9lc_xvgyc0000gn/T/ipykernel_16533/1940002320.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from_1300_end[['fr_text', 'nl_text']] = from_1300_end['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))
/var/folders/lh/rbpzcs3522qg45v9lc_xvgyc0000gn/T/ipykernel_16533/1940002320.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from_1300_end[['fr_text', 'nl_text']] = from_1300_end['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))


In [22]:
df_pdf_not_disponible["fr_text"] = "PDF not disponible"
df_pdf_not_disponible["nl_text"] = "PDF not disponible"
df_pdf_not_disponible.to_csv("pdf_not_disponible.csv")

/var/folders/lh/rbpzcs3522qg45v9lc_xvgyc0000gn/T/ipykernel_16533/4224887151.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pdf_not_disponible["fr_text"] = "PDF not disponible"
/var/folders/lh/rbpzcs3522qg45v9lc_xvgyc0000gn/T/ipykernel_16533/4224887151.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pdf_not_disponible["nl_text"] = "PDF not disponible"


In [7]:
df1300_end = pd.read_csv("./folder_to_ignore/processed_csv/from_1300_end.csv")
from2800_end = df1300_end.iloc[1500:]
from2800_end.to_csv("./processed_csv/from2800_end.csv")

In [ ]:
# df_pdf_disponible[['fr_text', 'nl_text']] = df_pdf_disponible['DocumentChambre'].apply(lambda url: pd.Series(get_text(url)))


In [ ]:
# def process_url(url):
#     left_text, right_text = get_text(url)
#     return (left_text, right_text)

# # Process URLs using ThreadPoolExecutor
# with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
#     results = list(executor.map(process_url, df_pdf_disponible['DocumentChambre']))

# # Unpack the results and add them as new columns to the DataFrame
# df_pdf_disponible[['left_column_clean', 'right_column_clean']] = pd.DataFrame(results, columns=['left', 'right'])

# # Display the updated DataFrame
# print(df_pdf_disponible)

In [ ]:
# df_pdf_not_disponible["fr_text"] = "PDF not disponible"
# df_pdf_not_disponible["nl_text"] = "PDF not disponible"
# df_pdf_disponible["fr_text"] = df_pdf_disponible["DocumentChambre"].map(lambda x: get_text(x)[0])
# df_pdf_disponible["nl_text"] = df_pdf_disponible["DocumentChambre"].map(lambda x: get_text(x)[1])

In [172]:
# df = pd.read_json('apercu_complet.json')

# def add_text_to_df(df):
#     df_pdf_not_disponible = df[df["DocumentChambre"].str.contains("pasdisponible")]
#     df_pdf_disponible = df[~df["DocumentChambre"].str.contains("pasdisponible")]
#     df_pdf_disponible["fr_text"] = df_pdf_disponible["DocumentChambre"].map(lambda x: get_text(x)[0])
#     df_pdf_disponible["nl_text"] = df_pdf_disponible["DocumentChambre"].map(lambda x: get_text(x)[1])
#     df_pdf_not_disponible["fr_text"] = "PDF not disponible"
#     df_pdf_not_disponible["nl_text"] = "PDF not disponible"

In [ ]:
# def new_get_text(df):
#     try:
#         rq = requests.get(df["DocumentChambre"])
#         left_column_text_all_pages = ""
#         right_column_text_all_pages = ""

#         with pdfplumber.open(io.BytesIO(rq.content)) as pdf:
#             for page in pdf.pages:
#                 words = page.extract_words()
#                 threshold_x = 300

#                 left_column_words = [word for word in words if word['x0'] < threshold_x]
#                 right_column_words = [word for word in words if word['x0'] >= threshold_x]

#                 left_column_text = ' '.join([word['text'] for word in left_column_words])
#                 right_column_text = ' '.join([word['text'] for word in right_column_words])

#                 left_column_text_all_pages += left_column_text + ' '
#                 right_column_text_all_pages += right_column_text + ' '

#                 left_column_clean = re.sub(r'(?<=\w)-(\s)(?=\w)', r'', left_column_text_all_pages) # remove end_line hyphens
#                 left_column_clean = re.sub(r'DOC 55 \d{4}/001', '', left_column_clean)
#                 left_column_clean = re.sub(r'\d{4}/001 DOC 55', '', left_column_clean)
#                 left_column_clean = re.sub(r'CHAMBRE \d+e SESSION DE LA 55e LÉGISLATURE \d{4}(?: \d+)?', '', left_column_clean)

#                 right_column_clean = re.sub(r'(?<=\w)-(\s)(?=\w)', r'', right_column_text_all_pages) # remove end_line hyphens
#                 right_column_clean = re.sub(r'DOC 55 \d{4}/001', '', right_column_clean)
#                 right_column_clean = re.sub(r'\d{4}/001 DOC 55', '', right_column_clean)
#                 right_column_clean = re.sub(r'\d{4} KAMER • \d+e ZITTING VAN DE 55e ZITTINGSPERIODE(?: \d+)?', '', right_column_clean)

#         df["fr_text"] = left_column_clean
#         df["nl_text"] = right_column_clean
#         return df

#     except Exception as e:
#         print(type(e))
#         return e

# for i in df_pdf_disponible.loc(): 
#     print(i)
#     # new_get_text(i)

# # search_links = get_search_url_list(end_page=334)

# # urls = list(itertools.chain.from_iterable(thread_map(get_urls_from_search_page, search_links)))

# # with requests.Session() as session:
#     # new_df = df for df in thread_map(partial(new_get_text, session=session), df_pdf_disponible)

# # properties.to_csv("properties.csv")